Nous allons nettoyer les données et récupérer les informations importantes.
## Nettoyage
Nous gardons les colonnes qui nous intéressent: 
- vendor
- description
- sales
- lvl
- trust
- member_since
- disputes_orders
et nous enlevons les valeurs NaN

In [1]:
import pandas as pd
import numpy as np

sellersDf = pd.read_csv('../../data/seller_duplicates_drop.csv', delimiter=";")
sellers = sellersDf.copy()
collums_to_drop =[ 'fe_allowed', 'percent_posfb',
       'fb_1Pos', 'fb_6Pos', 'fb_12Pos', 'fb_1Neu', 'fb_6Neu', 'fb_12Neu',
       'fb_1Neg', 'fb_6Neg', 'fb_12Neg', 'last', 'fb_left',
       'spendings', 'pgp', 'file', 'crawling_date']
sellers = sellers.drop(columns = collums_to_drop, axis=1)
sellers = sellers.drop(columns=['description'], axis=1)
sellers = sellers.dropna(subset=['trust'])
sellers['member_since'] =  pd.to_datetime(sellers['member_since'], format='%B %d, %Y')
sellers.head()

,vendor,sales,lvl,trust,member_since,disputes_orders
0,0ctopus,(342),Vendor level 5,Trust level 5,2019-04-28,0 / 0
1,0hp1,(1162),Vendor level 7,Trust level 7,2019-04-24,0 / 0
2,123pharma,(1268),Vendor level 5,Trust level 5,2020-03-22,0 / 0
3,1800empire,(214),Vendor level 4,Trust level 3,2020-01-10,0 / 0
4,19911,(1680),Vendor level 2,Trust level 1,2019-04-24,0 / 9


In [2]:

import re
def remove_parentheses(text):
       clean = re.match('\\((.*)\\)',str(text))
       return clean.groups()[0]


def remove_vendor_trst_level(text):
       clean = re.match('(.*)(\\d)',str(text))
       return clean.groups()[1]




sellers['lvl'] = sellers['lvl'].apply(remove_vendor_trst_level)
sellers['trust'] = sellers['trust'].apply(remove_vendor_trst_level)
sellers['sales'] = sellers['sales'].apply(remove_parentheses)
sellers.sales = sellers.sales.astype(int)
sellers.lvl = sellers.lvl.astype(int)
sellers.trust = sellers.trust.astype(int)
sellers.vendor = sellers.vendor.astype(str)
sellers.describe()


,sales,lvl,trust
count,2853.000000,2853.000000,2853.000000
mean,1091.132843,3.732562,3.441991
std,2844.433360,2.363744,2.431650
min,0.000000,0.000000,0.000000
25%,35.000000,1.000000,1.000000
50%,223.000000,4.000000,3.000000
75%,911.000000,6.000000,6.000000
max,40464.000000,9.000000,9.000000


Nous devons maintenant récupérer les vendeurs les plus populaires.

Nous allons utiliser une formule avec des coefficients complètements arbitraires.

$ 2*sales + 3*trust + 0.5*money\_made = Popularity $


In [3]:
import numpy as np
products_sells = pd.read_csv('../../data/list_duplicates_drop.csv', delimiter=";")
collums_to_drop = ['product_category', 'product_url', 'product_title',
       'product_id_subcategory', 'product_views_sales_quantityleft',
       'vendor_url', 'BTC', 'file', 'crawling_date']

products_sells = products_sells.drop(columns=collums_to_drop)
def cleanUsdColumn(text):
    return text.replace(',','')
products_sells =products_sells.dropna()
products_sells['USD'] = products_sells['USD'].apply(cleanUsdColumn)
products_sells['USD'] = products_sells['USD'].astype(float)
products_sells.head()

new_products_sells = products_sells.groupby('vendor').agg({'USD': ['sum']}).reset_index()

new_products_sells.columns = ['vendor', 'total']
new_products_sells.head()
new_products_sells.describe()
sellers = sellers.merge(new_products_sells, on='vendor')
sellers.to_csv(r'../../data/sellers.csv', index = False)

